In [2]:
import json
import random
import time
import requests
import re
import threading
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from pymongo import MongoClient
from bson.json_util import dumps, CANONICAL_JSON_OPTIONS

In [3]:
username = 'achang'
password = 'Pn123456'
monip = '192.168.1.113'
mondb = 'achang'
moncol='rent'
monclient = MongoClient(monip,
    username = username,
    password = password,
    authSource = mondb,
    authMechanism = 'SCRAM-SHA-256')
mondb = monclient[mondb]
monCol = mondb[moncol]

In [4]:
result = monCol.find({"$and":[{"縣市":"新北市"},{"性別要求":{"$in":["男女生皆可","男生"]}}]},{"_id":0})

In [5]:
list(result)

[{'縣市': '新北市',
  '鄉鎮': '三重區',
  '出租者': '查先生',
  '出租者身份': '屋主',
  '聯絡電話': '0918989993',
  '型態': '公寓',
  '現況': '獨立套房',
  '性別要求': '男女生皆可'},
 {'縣市': '新北市',
  '鄉鎮': '三重區',
  '出租者': '劉先生',
  '出租者身份': '屋主',
  '聯絡電話': '0922387388',
  '型態': '公寓',
  '現況': '分租套房',
  '性別要求': '男女生皆可'},
 {'縣市': '新北市',
  '鄉鎮': '新莊區',
  '出租者': '陳先生',
  '出租者身份': '代理人',
  '聯絡電話': '0922888227',
  '型態': '公寓',
  '現況': '雅房',
  '性別要求': '男女生皆可'},
 {'縣市': '新北市',
  '鄉鎮': '永和區',
  '出租者': '黃先生',
  '出租者身份': '屋主',
  '聯絡電話': '0986851077',
  '型態': '公寓',
  '現況': '獨立套房',
  '性別要求': '男女生皆可'},
 {'縣市': '新北市',
  '鄉鎮': '永和區',
  '出租者': '曾小姐',
  '出租者身份': '屋主',
  '聯絡電話': '0906288237',
  '型態': '公寓',
  '現況': '獨立套房',
  '性別要求': '男女生皆可'},
 {'縣市': '新北市',
  '鄉鎮': '淡水區',
  '出租者': '蕭先生',
  '出租者身份': '代理人',
  '聯絡電話': '0987361900',
  '型態': '公寓',
  '現況': '獨立套房',
  '性別要求': '男女生皆可'},
 {'縣市': '新北市',
  '鄉鎮': '中和區',
  '出租者': '莫先生',
  '出租者身份': '代理人',
  '聯絡電話': '0983801198',
  '型態': '公寓',
  '現況': '雅房',
  '性別要求': '男女生皆可'},
 {'縣市': '新北市',
  '鄉鎮': '中和區',
  '出租者': '莫先

In [36]:
dumps(result).decode('utf8')

AttributeError: 'str' object has no attribute 'decode'

In [2]:
def make_soup(href):
    # 建立各網頁的soup，以取得網頁資料結構
    headers = {
    'Connection': 'close',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'}
    res = requests.get(href, headers=headers)
    soup = bs(res.text, 'lxml')
    return soup

In [3]:
def contact_phone(soup):
    if len(soup.select('div.hidtel')[0].text) != 0:
        if len(soup.select('div.hidtel')[0].text) > 11:
            return soup.select('div.hidtel')[0].text.split(' ')[0].replace('-','')
        else:
            return soup.select('div.hidtel')[0].text
    else:
        return soup.select('span.dialPhoneNum')[0].get('data-value').split(' ')[0].replace('-','')

In [4]:
soup = make_soup('https://rent.591.com.tw/rent-detail-9345031.html')
contact_phone(soup)

IndexError: list index out of range

In [29]:
def chrome_driver():
    chrome_options = Options()
    prefs = {"profile.default_content_setting_values.notifications":2}
    chrome_options.add_experimental_option("prefs",prefs)
    chrome_options.add_argument('user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36"')
    chrome_options.add_argument('--headless')
    driver = webdriver.Chrome('/Users/afrithero/Desktop/面試題/chromedriver',options=chrome_options)
    return driver

In [37]:
link_url = []
driver = chrome_driver()
driver.get('https://rent.591.com.tw/?kind=0&region=4')
driver.refresh()
for j in driver.find_elements_by_xpath('//*[@id="content"]/ul/li/h3/a'):
    print(j.text)
    link_url.append(j.get_attribute('href'))
driver.close()

近博愛國小，環境清幽，交通便利
快!!信義區全新酒店式公寓/最後一間!
全坤威峰豪宅2+1房
圓山捷運5分鐘可短期一個月
南軟、中研院刷卡套房私人洗衣機
近台北車站24H安全監控禁菸
典美套房全套系統傢俱
台北橋圓山捷運站，車位電梯2房2廳附家具
西門大陽台採光通風良好，可炊精緻雅房
萬大路陽光美屋
考試院世新公寓首選，富山幸運宅溫馨3房
靠近古亭捷運全新套房
中山站乾淨安全交通便利套房
套房•全套系統傢俱
市府站旁近松菸湖畔獨立套房
ServiceApartment可月租
濱湖皇家湖畔分租雅房8000起
離捷運走路只要3分鐘~鬧中取靜
安和捷運靜巷內,新成屋3+1房,百萬裝潢
新★士林捷運一房一廳一衛★超乾淨超大空間
台北火車站崗石飯店宅@博愛路漢口街桂林路
230號10樓之1遠東雙星大廈住辦套房
近捷運3分鐘，1房1廳採光通風佳，可短租
可短租華山溫馨二房一廳定期免費打掃房間
獨立專屬洗衣機,全新百萬裝潢
陶淵明天母美國日本學校面磺溪景觀飯店豪宅
光華商場套房(近北科大)
捷運中山站套房
全新裝潢頂加住家
捷運龍山寺站一分鐘，陽光溫馨房~


In [31]:
link_url

['https://rent.591.com.tw/rent-detail-9362312.html',
 'https://rent.591.com.tw/rent-detail-9402144.html',
 'https://rent.591.com.tw/rent-detail-9379694.html',
 'https://rent.591.com.tw/rent-detail-6287529.html',
 'https://rent.591.com.tw/rent-detail-9298278.html',
 'https://rent.591.com.tw/rent-detail-9345910.html',
 'https://rent.591.com.tw/rent-detail-9358081.html',
 'https://rent.591.com.tw/rent-detail-9342963.html',
 'https://rent.591.com.tw/rent-detail-9389821.html',
 'https://rent.591.com.tw/rent-detail-9396679.html',
 'https://rent.591.com.tw/rent-detail-9382397.html',
 'https://rent.591.com.tw/rent-detail-9315969.html',
 'https://rent.591.com.tw/rent-detail-9275768.html',
 'https://rent.591.com.tw/rent-detail-9293949.html',
 'https://rent.591.com.tw/rent-detail-9293373.html',
 'https://rent.591.com.tw/rent-detail-9346520.html',
 'https://rent.591.com.tw/rent-detail-9327980.html',
 'https://rent.591.com.tw/rent-detail-9370263.html',
 'https://rent.591.com.tw/rent-detail-9371589.

In [28]:
soup = make_soup('https://rent.591.com.tw/?kind=0&region=4')
soup.select('h3')

[<h3>
 <a href="//rent.591.com.tw/rent-detail-9379694.html" target="_blank">獨立大套房，有開窗</a>
 <span class="labelOne">黄金曝光</span>
 </h3>, <h3>
 <a href="//rent.591.com.tw/rent-detail-6287529.html" target="_blank">民生社區-7坪巧美寧靜套房</a>
 <span class="labelOne">黄金曝光</span>
 </h3>, <h3>
 <a href="//rent.591.com.tw/rent-detail-9298278.html" target="_blank">坡平車位忠孝東路四段,近國父紀念館捷運站</a>
 <span class="labelOne">黄金曝光</span>
 </h3>, <h3>
 <a href="//rent.591.com.tw/rent-detail-9345910.html" target="_blank">捷運中山站善導寺站松江南京站兩房一廳</a>
 <span class="labelOne">黄金曝光</span>
 </h3>, <h3>
 <a href="//rent.591.com.tw/rent-detail-9358081.html" target="_blank">優質套房近捷運站另可短租</a>
 <span class="labelOne">黄金曝光</span>
 </h3>, <h3>
 <a href="//rent.591.com.tw/rent-detail-9342963.html" target="_blank">捷運1分.新落成電梯庭院40吋液晶乾溼分</a>
 <span class="labelOne">黄金曝光</span>
 </h3>, <h3>
 <a href="//rent.591.com.tw/rent-detail-9389821.html" target="_blank">大安捷運5分鐘鳥語花香一房一廳兩張雙人床</a>
 <span class="labelOne">黄金曝光</span>
 </h3>, <h3>
 <a href="//re

In [22]:
username = 'achang'
password = 'Pn123456'
monip = "192.168.1.113"
mondb = "achang"
moncol='591'

def monCon(username,password,monip,mondb,moncol):
    monclient = MongoClient(monip,
                username = username,
                password = password,
                authSource = mondb,
                authMechanism = 'SCRAM-SHA-256')
    mondb = monclient[mondb]
    momCol = mondb[moncol]
    return momCol

def monInsert(username,password,monip,mondb,moncol,data):
    collection = monCon(username,password,monip,mondb,moncol)
    collection.insert_many(data,ordered=False)
    
def monFindAlldf(username,password,monip,mondb,moncol):
    momCol = monCon(username,password,monip,mondb,moncol)
    mondf = pd.DataFrame(list(momCol.find({})))
    return mondf

In [23]:
data = [{'物件編號': 'R9377114', '縣市': '台北市', '鄉鎮': '北投區', '出租者': '黃先生', '出租者身份': '屋主', '聯絡電話': '0986851089', '型態': '公寓', '現況': '整層住家', '性別要求': None}, {'物件編號': 'R9340073', '縣市': '台北市', '鄉鎮': '中山區', '出租者': '陳小姐', '出租者身份': '屋主', '聯絡電話': '0986851077', '型態': '公寓', '現況': '整層住家', '性別要求': None}, {'物件編號': 'R9338738', '縣市': '台北市', '鄉鎮': '中正區', '出租者': '龔先生', '出租者身份': '代理人', '聯絡電話': '0907007685', '型態': '公寓', '現況': '整層住家', '性別要求': None}]

In [24]:
monInsert(username,password,monip,mondb,moncol,data)